# Создание агента для работы с функциями GigaChat

В раделе приводится пример создания агента, который использует функции GigaChat.

:::note

Подробнее о функциях — в разделе документации GigaChat API [Работа с функциями](https://developers.sber.ru/docs/ru/gigachat/api/function-calling).

:::

Итоговый агент будет работать с тремя функциями:

* расчета расстояния;
* отправки SMS-сообщения;
* поиска фильмов.

Также в этом ноутбуке показаны доп. примеры функций

## Подготовка к разработке

Установка библиотеки для работ с GigaChat

In [ ]:
!pip install -U langchain_gigachat langgraph -q


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


Перед разработкой агента инициализируйте GigaChat.

Для лучшего результата в примере используется модель GigaChat-Pro или Max.
Вы можете использвать любую из доступных [моделей](https://developers.sber.ru/docs/ru/gigachat/models).

Получить ключ для доступа к GigaChat API вы можете на сайте [developers.sber.ru](https://developers.sber.ru)

In [ ]:
# В случае, если вы используете файл .env для настройки авторизации, раскомментируйте следующие строки
# !pip install python-dotenv -q
# from dotenv import find_dotenv, load_dotenv
# load_dotenv(find_dotenv())

True

In [ ]:
from langchain_gigachat.chat_models import GigaChat

giga = GigaChat(
    credentials="ВАШ AUTHORIZATION KEY", model="GigaChat-Pro", verify_ssl_certs=False
)

## Создание функции

Для создания и поисания функций, которые сможет вызывать модель используйте декоратор `@tool`.

:::tip

Модели GigaChat значительно лучше работают с функциями, которые описаны согласно приведенным примерам.
При описании функции уделяйте внимание подробному описанию структуры входных и выходных данных, не забывайте указывать краткое описание самой функции и примеры ее использования.

[Ниже](#functions-descriptions-example) вы найдете несколько примеров хорошо описанных функций.

:::

### Описание функции отправки СМС-сообщений

Декоратор `@giga_tool` является расширением декоратора `@tool` из LangChain, но добавляет к нему некоторые дополнительные возможнсоти, например возможность описать примеры использования функции с помощью few-shot.

In [10]:
from langchain_gigachat.tools.giga_tool import giga_tool
from pydantic import BaseModel, Field


class SendSmsResult(BaseModel):
    status: str = Field(description="Статус отправки сообщения")
    message: str = Field(description="Сообщение о результате отправки SMS")


few_shot_examples = [
    {
        "request": "Можешь ли ты отправить SMS-сообщение на номер 123456789 с содержимым 'Привет, как дела?'",
        "params": {"recipient": "123456789", "message": "Привет, как дела?"},
    }
]


@giga_tool(few_shot_examples=few_shot_examples)
def send_sms(
    recipient: str = Field(description="Номер телефона получателя"),
    message: str = Field(description="Содержимое сообщения"),
) -> SendSmsResult:
    """Отправить SMS-сообщение"""
    print(f"! send_sms to {recipient}, text: {message}")
    # Здесь должна быть реальная отправка СМС через внешний шлюз
    return SendSmsResult(status="OK", message="Сообщение отправлено!")


## Создание агента

Для инициализации агента используйте метод `create_gigachat_functions_agent(model, tools)`, который принимает на вход объект модели и массив функций, которые сможетвызывать модель.

In [16]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langgraph.prebuilt import create_react_agent

functions = [send_sms]
giga_with_functions = giga.bind_functions(functions)

agent_executor = create_react_agent(giga_with_functions, functions)
resp = agent_executor.invoke({"messages": [HumanMessage(content="Отправь смс с текстом привет на номер 2223334445")]})
resp['messages'][-1].content

! send_sms to 2223334445, text: привет


'Сообщение с текстом "привет" было успешно отправлено на номер 2223334445.'

Теперь создадим агента с памятью и метод для ведения чатов

In [24]:
import time
from langgraph.checkpoint.memory import MemorySaver

agent_executor = create_react_agent(giga_with_functions, 
                                    functions, 
                                    checkpointer=MemorySaver(),
                                    state_modifier="""Ты бот для отправки смс. 
Спроси у пользователя все нужные данные перед отправкой.""")

def chat(agent_executor, thread_id: str):
    config = {"configurable": {"thread_id": thread_id}}

    while True:
        user_input = input("Клиент: ")
        if user_input == "":
            break
        print(f"User: {user_input}")
        resp = agent_executor.invoke({"messages": [HumanMessage(content=user_input)]}, config=config)
        bot_answer = resp['messages'][-1].content
        print("\033[93m" + f"Bot: {bot_answer}" + "\033[0m")
        time.sleep(1) # Fix bug in Jupyter Notebook for VSCode

chat(agent_executor, "id_1")

User: Привет
Bot: Здравствуйте! Как я могу Вам помочь сегодня?
User: Что ты умеешь?
Bot: Я могу отправлять SMS-сообщения. Могу помочь Вам с этим?
User: Хочу отправить смс
Bot: Конечно, я могу помочь с этим. Не могли бы Вы предоставить мне номер получателя и сообщение, которое хотите отправить?
User: На номер 111222333444
Bot: Пожалуйста, укажите текст сообщения, который вы хотите отправить на этот номер.
User: Сообщение - Всем привет!
! send_sms to 111222333444, text: Всем привет!
Bot: Ваше сообщение "Всем привет!" было успешно отправлено на номер 111222333444.


## Работа с несоклькими функциями

Опишите еще две функции: для поиска фильмов и для расчета расстояния.

### Функция поиска фильмов

В примере приводится функцися посика фильмов на основе некторых параметров.

In [26]:
from typing import List, Optional


class SearchMoviesResult(BaseModel):
    movies: List[str] = Field(
        description="Список названий фильмов, соответствующих заданным критериям поиска"
    )


few_shot_examples = [
    {"request": "Найди все фильмы жанра комедия", "params": {"genre": "комедия"}}
]


@giga_tool(few_shot_examples=few_shot_examples)
def search_movies(
    genre: Optional[str] = Field(None, description="Жанр фильма"),
    year: Optional[int] = Field(None, description="Год выпуска фильма"),
    actor: Optional[str] = Field(None, description="Имя актера, снимавшегося в фильме"),
) -> SearchMoviesResult:
    """Поиск фильмов на основе заданных критериев"""
    print(f"! search_movies genre {genre}, year: {year}, actor: {actor}")
    # Здесь должна быть реальная отправка СМС через внешний шлюз
    return SearchMoviesResult(movies=["Опенгеймер"])

### Функция расчета расстояний

В примере приводится функция, которая вычисляет расстояние между заданными начальным и конечным пуктами.

In [29]:
class TripDistanceResult(BaseModel):
    distance: int = Field(
        description="Расстояние между начальным и конечным местоположением в километрах"
    )


few_shot_examples = [
    {
        "request": "Насколько далеко от Москвы до Санкт-Петербурга?",
        "params": {"start_location": "Москва", "end_location": "Санкт-Петербург"},
    }
]


@giga_tool(few_shot_examples=few_shot_examples)
def calculate_trip_distance(
    start_location: str = Field(description="Начальное местоположение"),
    end_location: str = Field(description="Конечное местоположение"),
) -> TripDistanceResult:
    """Рассчитать расстояние между двумя местоположениями."""
    print(
        f"! calculate_trip_distance start_location {start_location}, end_location: {end_location}"
    )
    # Здесь должна быть реальная отправка СМС через внешний шлюз
    return TripDistanceResult(distance=650)

Передайте функции в агент и обратитесь к нему с подходящими запросами.

In [ ]:
functions = [send_sms, search_movies, calculate_trip_distance]
giga_with_functions = giga.bind_functions(functions)
agent_executor = create_react_agent(giga_with_functions, 
                                    functions, 
                                    checkpointer=MemorySaver())

chat(agent_executor, thread_id="id_321")

User: Найди фильмы в жанре биография 2023 года
! search_movies genre биография, year: 2023, actor: annotation=Union[str, NoneType] required=False default=None description='Имя актера, снимавшегося в фильме'
Bot: В 2023 году вышел фильм в жанре биографии под названием "Опенгеймер".
User: Расстояние от Москвы до Питера
! calculate_trip_distance start_location Москва, end_location: Санкт-Петербург
Bot: Расстояние по дороге от Москвы до Санкт-Петербурга составляет примерно 650 километров.


## Функция реакций пользователя на проигрываемый контент
В этом примере рассмотрим создание тулов, через класс BaseTool.
Через наследование класса BaseTool, можно реализовывать тулы с async выполнением.

In [45]:
from typing import Literal, Optional, Type

from langchain_core.callbacks import CallbackManagerForToolRun
from langchain_core.tools import BaseTool
from langchain_gigachat.tools.giga_tool import FewShotExamples


class PlayerReactionsInput(BaseModel):
    reaction: Literal["add_like", "remove_like", "add_dislike"] = Field(
        description="""Действие, которое необходимо выполнить:
- add_like - поставить лайк и добавить в избранное, коллекцию, подборку;
- remove_like - удалить лайк и удалить из избранного, коллекции, подборки, при вызове текущий аудиоконтент переключается на следующий;
- add_dislike - поставить дизлайк, удалить из избранного, коллекции, подборки и переключить музыкальный контент на слудующий.
Параметр add_dislike устанавливается только если пользователь явно проявляет негативную реакцию. Если он просто просит пропустить трек или включить следующий, то это не является реакцией и функцию обработки реакций вызывать не нужно."""  # noqa
    )
    content_type: Optional[
        Literal[
            "track",
            "artist",
            "album",
            "playlist",
            "release",
            "podcast",
            "podcast_episode",
            "audiobook",
            "audiobook_chapter",
        ]
    ] = Field(
        description=(
            "Тип музыкального контента, к которому применяется действие в соответствии с запросом пользователя. "  # noqa
            "Заполняется только в случае, если пользователь явно указал тип контента."
        )
    )


class PlayerReactionsOutput(BaseModel):
    status: Literal["success", "fail"] = Field(
        description="Статус - удалось ли совершить запрошенное пользователем действие"
    )
    error: Optional[str] = Field(
        description="Текст ошибки в случае, если status == fail", default=None
    )


class PlayerReactionsTool(BaseTool):
    name: str = "player_reactions"
    description: str = (
        "Функция обработки реакций пользователя на проигрываемый контент. "
        "Функция вызывается только в случае, если на устройстве пользователя играет звуковой контент.\n"  # noqa
        "В случае положительной реакции на проигрываемый контент, он продолжит проигрываться.\n"  # noqa
        "В случае отрицательной реакции он будет переключен на следующий аналогичный по типу. "  # noqa
        "В своем ответе сообщи об этом пользователю.\n"
        "После вызова этой функции отвечай в соответствии со своим характером, "
        "разнообразно, но коротко (не более 7 слов) и не задавай вопросы пользователю, "
        "так как он будет продолжать слушать текущий или иной контент.\n"
        "Если пользователь уже оставлял реакцию на проигрываемый контент, "
        "то информация об этом будет возвращена в поле error. "
        "В случае ошибки отвечай в соответствии со своим характером и текстом из поля error."  # noqa
    )
    args_schema: Type[BaseModel] = PlayerReactionsInput
    return_schema: Type[BaseModel] = PlayerReactionsOutput
    few_shot_examples: FewShotExamples = [
        {"request": "Лайк", "params": {"reaction": "add_like"}},
        {"request": "Дизлайк", "params": {"reaction": "add_dislike"}},
        {"request": "убери лайк", "params": {"reaction": "remove_like"}},
        {"request": "добавь в плейлист", "params": {"reaction": "add_like"}},
        {"request": "удали из плейлиста", "params": {"reaction": "remove_like"}},
        {
            "request": "Добавь артиста в избранное",
            "params": {"reaction": "add_like", "content_type": "artist"},
        },
        {
            "request": "Мне нравится это альбом",
            "params": {"reaction": "add_like", "content_type": "album"},
        },
    ]

    def _run(
        self,
        reaction: str,
        content_type: Optional[str] = None,
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> PlayerReactionsOutput:
        """Use the tool."""
        print(f"! player_reaction reaction {reaction}, content_type: {content_type}")
        if reaction == "add_dislike":
            return PlayerReactionsOutput(status="fail", error="Дизлайки пока не поддерживаются")
        return PlayerReactionsOutput(status="success")

In [46]:
functions = [PlayerReactionsTool()]
giga_with_functions = giga.bind_functions(functions)
agent_executor = create_react_agent(giga_with_functions, 
                                    functions, 
                                    checkpointer=MemorySaver(), debug=False)

chat(agent_executor, thread_id="id_324")

User: Лайк этому треку
! player_reaction reaction add_like, content_type: track
Bot: Ваш лайк зафиксирован! Трек продолжает играть.
User: Поставь дизлайк текущему альбому
! player_reaction reaction add_dislike, content_type: album
Bot: Ой, что-то пошло не так. Похоже, дизлайки для альбомов сейчас недоступны. Может, попробуем еще раз?


## Другие примеры тулов (без запуска)
### Получение напоминаний

In [49]:
# Комментарий к классу подкладывается в JSON формирующийся для функций
# в поле "description"
class Reminder(BaseModel):
    """Метаинформация напоминания."""

    id: Optional[str] = Field(description="Идентификатор напоминания.", default=None)
    cron: Optional[str] = Field(
        description=(
            "Описание периодичности напоминания. "
            "Здесь будет передано человекочитаемое описание переодичности напоминания. "
            "Если поле отсутствует, то у напоминания нет периодичности (единоразовое)."
        ), default=None
    )
    title: Optional[str] = Field(
        description="Название напоминания, о чем надо напомнить.", default=None
    )
    devices: Optional[List[str]] = Field(
        description="Словарь устройств, к которым привязаны напоминания", default=None
    )
    reminderTime: Optional[str] = Field(description="Дата и время старта напоминания.", default=None)
    createdAt: Optional[str] = Field(description="Дата и время создания напоминания.", default=None)


class GetReminderInput(BaseModel):
    title: Optional[str] = Field(description="Текст напоминания", default=None)
    date_time: Optional[str] = Field(
        description="Относительное время и дата напоминания на русском языке", default=None
    )
    device_name: Optional[str] = Field(
        description="Название устройства, на котором следует проверить напоминание", default=None
    )
    room: Optional[str] = Field(
        description="Название комнаты в которой следует проверить напоминание", default=None
    )


class GetReminderOutput(BaseModel):
    """Ответ на get_reminder"""

    status: Literal["success", "fail"] = Field(
        description="Статус - удалось ли найти список установленных напоминаний", default=None
    )
    error: Optional[str] = Field(
        description="Текст ошибки в случае, если status == fail", default=None
    )
    items: Optional[List[Reminder]] = Field(
        description=(
            "Список установленных напоминаний. "
            "В списке перечислены идентификаторы напоминаний (id), "
            "дата и время старта напоминания (reminderTime), "
            "периодичность напоминания в человекочитаемом формате (cron), "
            "название напоминания (title), "
            "дата и время создания напоминания (createdAt)."
        ), default=None
    )


class GetReminderTool(BaseTool):
    name: str = "get_reminder"
    description: str = (
        "Получить метаинформацию обо всех установленных напоминаниях. "  # noqa
        "Вызови эту функцию перед удалением или изменением напоминаний, чтобы получить id напоминаний. "  # noqa
        "В случае если пользователь хочет удалить или изменить напоминание и в контексте диалога нет необходимых id, "  # noqa
        "то сначала вызови эту функцию для получения идентификатора id и ответь пустым сообщением, "  # noqa
        "а далее при необходимости вызови следующую функцию для выполнения запроса пользователя.\n"  # noqa
        "После вызова данной функции ответь пользователю в следующем стиле: "  # noqa
        '"У вас установлено 2 напоминания. Через 10 минут выключить духовку на кухне, а завтра в 3 часа сходить в гости.'  # noqa
    )
    args_schema: Type[BaseModel] = GetReminderInput
    return_schema: Type[BaseModel] = GetReminderOutput
    few_shot_examples: FewShotExamples = [
        {"request": "мои напоминания", "params": {}},
        {"request": "удали напоминалку на завтра в пять", "params": {}},
        {
            "request": "перенеси напоминание поздравить маму на шесть вечера",
            "params": {},
        },
        {"request": "какое у меня количество напоминаний", "params": {}},
        {"request": "озвучь напоминалки", "params": {}},
    ]

    def _run(
        self,
        title: Optional[str] = None,
        date_time: Optional[str] = None,
        device_name: Optional[str] = None,
        room: Optional[str] = None,
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> GetReminderOutput:
        """Логика тула"""
        pass

Проверяем схему, которая генерируется для API GigaChat

In [51]:
import json

from langchain_gigachat.utils.function_calling import convert_to_gigachat_function

print(
    json.dumps(
        convert_to_gigachat_function(GetReminderTool()), indent=2, ensure_ascii=False
    )
)

{
  "name": "get_reminder",
  "description": "Получить метаинформацию обо всех установленных напоминаниях. Вызови эту функцию перед удалением или изменением напоминаний, чтобы получить id напоминаний. В случае если пользователь хочет удалить или изменить напоминание и в контексте диалога нет необходимых id, то сначала вызови эту функцию для получения идентификатора id и ответь пустым сообщением, а далее при необходимости вызови следующую функцию для выполнения запроса пользователя.\nПосле вызова данной функции ответь пользователю в следующем стиле: \"У вас установлено 2 напоминания. Через 10 минут выключить духовку на кухне, а завтра в 3 часа сходить в гости.",
  "parameters": {
    "description": "Получить метаинформацию обо всех установленных напоминаниях. Вызови эту функцию перед удалением или изменением напоминаний, чтобы получить id напоминаний. В случае если пользователь хочет удалить или изменить напоминание и в контексте диалога нет необходимых id, то сначала вызови эту функцию

### Изменение напоминаний

In [53]:
class ChangeReminderInput(BaseModel):
    id: str = Field(description="id напоминания")
    title: Optional[str] = Field(description="Новый текст напоминания")
    date_time: Optional[str] = Field(
        description="Новые время и дата напоминания на русском языке. "
        "Передай только то, что сказал пользователь, не меняя формат."
    )
    device_name: Optional[str] = Field(
        description="Новое название устройства, на которое следует поставить напоминание"  # noqa
    )


class ChangeReminderOutput(BaseModel):
    status: Literal["success", "fail"] = Field(
        description="Статус - удалось ли найти список установленных напоминаний"
    )
    error: Optional[str] = Field(
        description="Текст ошибки в случае, если status == fail"
    )
    reminder: Optional[Reminder] = Field(description="Параметры созданного напоминания")


class ChangeReminderTool(BaseTool):
    name: str = "change_reminder"
    description: str = (
        "Изменить напоминание по id.\n"
        "Если пользователь просит изменить напоминание, "
        "но не указывает какое и какие изменения надо внести, "
        "то в ответе попроси предоставить дополнительную информацию.\n"
        "Если просит изменить напоминание и не указывает какое, "
        "но указывает какие изменения внести, "
        "то сначала получи метаинформацию о напоминаниях, "
        "вызвав нужную функцию, "
        "перечисли их в ответе и уточни какое из них изменить.\n"
        "Если просит изменить напоминание, "
        "указывая какое, но не указывая изменения, "
        "то сначала получи метаинформацию обо всех напоминаниях, "
        "вызвав нужную функцию, "
        "перечисли их в ответе и при наличии id, "
        "соответствующего запросу, уточни какие изменения надо внести.\n"
        "Если просит изменить напоминание, "
        "указывая какое и какие изменения внести, "
        "то получи метаинформацию обо всех напоминаниях, "
        "вызвав нужную функцию, и при наличии id, "
        "соответствующего запросу пользователя, вызови функцию изменения напоминаня по id.\n\n"  # noqa
        "Вызывай данную функцию только при наличии нужного id и информации о том как надо изменить напоминание."  # noqa
    )
    args_schema: Type[BaseModel] = ChangeReminderInput
    return_schema: Type[BaseModel] = ChangeReminderOutput
    few_shot_examples: FewShotExamples = [
        {
            "request": "Изменить напоминание с id 123 на сегодня в 19 30",
            "params": {"id": "123", "date_time": "сегодня в 19 30"},
        }
    ]

    def _run(
        self,
        id: str,  # noqa
        title: Optional[str] = None,
        date_time: Optional[str] = None,
        device_name: Optional[str] = None,
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> ChangeReminderOutput:
        """Логика тула"""
        pass

Проверяем схему, которая генерируется для API GigaChat

In [54]:
print(
    json.dumps(
        convert_to_gigachat_function(ChangeReminderTool()), indent=2, ensure_ascii=False
    )
)

{
  "name": "change_reminder",
  "description": "Изменить напоминание по id.\nЕсли пользователь просит изменить напоминание, но не указывает какое и какие изменения надо внести, то в ответе попроси предоставить дополнительную информацию.\nЕсли просит изменить напоминание и не указывает какое, но указывает какие изменения внести, то сначала получи метаинформацию о напоминаниях, вызвав нужную функцию, перечисли их в ответе и уточни какое из них изменить.\nЕсли просит изменить напоминание, указывая какое, но не указывая изменения, то сначала получи метаинформацию обо всех напоминаниях, вызвав нужную функцию, перечисли их в ответе и при наличии id, соответствующего запросу, уточни какие изменения надо внести.\nЕсли просит изменить напоминание, указывая какое и какие изменения внести, то получи метаинформацию обо всех напоминаниях, вызвав нужную функцию, и при наличии id, соответствующего запросу пользователя, вызови функцию изменения напоминаня по id.\n\nВызывай данную функцию только при н

### Удаление напоминаний

In [56]:
class DeleteReminderInput(BaseModel):
    ids: List[str] = Field(
        description="Список идентификаторов id напоминаний, которые нужно удалить"
    )


class DeleteReminderOutput(BaseModel):
    """Ответ на delete_reminder"""

    status: Literal["success", "fail"] = Field(
        description="Статус - удалось ли удалить напоминание."
    )
    error: Optional[str] = Field(
        description="Текст ошибки в случае, если status == fail", default=None
    )


class DeleteReminderTool(BaseTool):
    name: str = "delete_reminder"
    description: str = (
        "Удалить напоминания по id. "
        "Если пользователь явно не передал id напоминания, "
        "то получи метаинформацию о напоминаниях, "
        "вызвав сначала соответствующую функцию, "
        "и только затем используй функцию удаления напоминания по id.\n"
        "Если в контексте беседы с пользователем у тебя есть необходимый id, "
        "то перед запуском этой функции тебе необходимо переспросить пользователя "
        "точно ли он хочет удалить данное напоминание и только после согласия удалять. "
        "Если пользователь просит удалить все напоминания "
        "и в контексте диалога есть необходимые id или пользователь явно передает id напоминания,"
        " которое надо удалить, то вызови эту функцию, переспрашивать пользователя не нужно."
        " В остальных случаях, при наличии необходимых id в контексте диалога "
        "и готовности удалить напоминание, сначала переспроси пользователя"
        " подтверждает ли он удаление напоминания и вызывай функцию"
        " только при наличии подтверждения от пользователя."
    )
    args_schema: Type[BaseModel] = DeleteReminderInput
    return_schema: Type[BaseModel] = DeleteReminderOutput
    few_shot_examples: FewShotExamples = []

    def _run(
        self,
        ids: List[str],
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> DeleteReminderOutput:
        """Логика тула"""
        pass

In [57]:
print(
    json.dumps(
        convert_to_gigachat_function(DeleteReminderTool()), indent=2, ensure_ascii=False
    )
)

{
  "name": "delete_reminder",
  "description": "Удалить напоминания по id. Если пользователь явно не передал id напоминания, то получи метаинформацию о напоминаниях, вызвав сначала соответствующую функцию, и только затем используй функцию удаления напоминания по id.\nЕсли в контексте беседы с пользователем у тебя есть необходимый id, то перед запуском этой функции тебе необходимо переспросить пользователя точно ли он хочет удалить данное напоминание и только после согласия удалять. Если пользователь просит удалить все напоминания и в контексте диалога есть необходимые id или пользователь явно передает id напоминания, которое надо удалить, то вызови эту функцию, переспрашивать пользователя не нужно. В остальных случаях, при наличии необходимых id в контексте диалога и готовности удалить напоминание, сначала переспроси пользователя подтверждает ли он удаление напоминания и вызывай функцию только при наличии подтверждения от пользователя.",
  "parameters": {
    "description": "Удалить н

## Смотрите также

* [Работа с функциями](https://developers.sber.ru/docs/ru/gigachat/api/function-calling)